# DC Circuit Analysis with Dependent Sources

## Simulating Resistor, Independent voltage and current sources, dependent sources

### Circuit with Current Controlled Current Source

In [1]:
import schemdraw as schem
import schemdraw.elements as elm

d = schem.Drawing()

Line1 = d.add(elm.LINE, d='right')
d += elm.Line().right().length(3)
d += elm.SourceI().up().label(('5A'), loc='bottom')
d += elm.Line().left().length(2)
d.push()
d += elm.Resistor().down().label('4A')
d.pop()
d += elm.Line().up().length(2)
d += elm.SourceControlledI().left().label('$3i_1$')
d += elm.Line().down().length(2)
d.push()
d += elm.Resistor().label('2Ω').right()
d.pop()
d.push()
d += elm.Resistor().label('8Ω').down()
d.pop()
d += elm.Resistor().label('6Ω').left()
d += elm.SourceV().label('50V').down().reverse()
d += elm.Line().right()
d += elm.Ground()

d

d.save('ccvs.png')


In [2]:
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import IntSlider, link, VBox, Tab
# import ipywidgets as widgets
from IPython.display import display, clear_output
from ipysheet import sheet, cell, row, column, cell_range

sheet1 = sheet(rows=5, columns=15, column_headers=False, row_headers=False)

R1 = IntSlider(
         value=1,
         min=0,
         max=20,
         step=1,
        orientation='vertical')

R2 = IntSlider(
         value=1,
         min=0,
         max=20,
         step=1,
        orientation='vertical')

R3 = IntSlider(
         value=1,
         min=0,
         max=20,
         step=1,
        orientation='vertical')

R4 = IntSlider(
         value=1,
         min=0,
         max=20,
         step=1,
        orientation='vertical')

I = IntSlider(
         value=1,
         min=0,
         max=20,
         step=1,
        orientation='vertical')

V1 = IntSlider(
         value=1,
         min=0,
         max=50,
         step=1,
        orientation='vertical')

F = IntSlider(
         value=1,
         min=0,
         max=20,
         step=1,
        orientation='vertical')


box = VBox([R1])
# defining a list with the contents of our windows
children = [VBox([R1]), VBox([R2]), VBox([R3]), VBox([R4]), VBox([I]), VBox([V1]), VBox([F])]
# initializing a tab
tab = Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'R1')
tab.set_title(1, 'R2')
tab.set_title(2, 'R3')
tab.set_title(3, 'R4')
tab.set_title(4, 'I')
tab.set_title(5, 'V1')
tab.set_title(6, 'F')
tab

In [11]:
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *


In [12]:
circuit = Circuit('Circuit with Controlled Current Voltage Source')
circuit.R(1, 2, 3, R1.value@u_Ω)
circuit.R(2, 3, circuit.gnd, R2.value@u_Ω)
circuit.R(3, 3, 4,            R3.value@u_Ω)
circuit.R(4, 4, circuit.gnd, R4.value@u_Ω)
circuit.V(1, 1, circuit.gnd, V1.value@u_V)
circuit.V('test', 1, 2,       0@u_V)
circuit.I(1, circuit.gnd, 4, I.value@u_A)
circuit.F(1, 4, 3, 'Vtest', F.value)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.operating_point()

button_send = widgets.Button(
                description='Simulate Circuit',
                tooltip='Simulate',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        with open('readings.txt', 'a') as f:
            circuit_readings = []
            for node in analysis.nodes.values():
                circuit_readings.append('Node {}: {:4.1f} V'.format(str(node), float(node)))
                print('Node {}: {:4.1f} V'.format(str(node), float(node)))   
            f.write(str(circuit_readings))
            f.write('\n')
        for node in analysis.branches.values():
            print('Node {}: {:5.2f} A'.format(str(node), float(node)))


button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

vbox_result



In [30]:
lines = []
with open('readings.txt') as f:
    lines = f.readlines()
count = 0
for line in lines:
    count += 1
    print(line)  

['Node 1:  1.0 V', 'Node 4:  0.7 V', 'Node 3:  0.7 V', 'Node 2:  1.0 V']

['Node 1:  1.0 V', 'Node 4:  8.4 V', 'Node 3:  1.3 V', 'Node 2:  1.0 V']

